# Model-Specific Input Data Preprocessing in CONFLUENCE

## Introduction

This notebook covers the model-specific preprocessing steps for input data in CONFLUENCE. After completing the model-agnostic preprocessing, we now focus on tailoring our data to the specific requirements of the chosen hydrological model (e.g., SUMMA, HYPE, or MESH).

Key aspects covered in this notebook include:

1. Formatting data according to the chosen model's input specifications
2. Generating model-specific configuration files
3. Preparing initial conditions and parameter files
4. Creating forcing data in the required format and resolution

In this notebook we ensure that our preprocessed data is compatible with the chosen hydrological model. By the end of this process, you will have a complete set of input files ready for model initialization and simulation.

## First we import the libraries and functions we need

In [1]:
import sys
from pathlib import Path
from typing import Dict, Any
import logging
import yaml # type: ignore

current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
sys.path.append(str(parent_dir))

#from utils.dataHandling_utils.specificPreProcessor_util import SummaPreProcessor_spatial, flashPreProcessor # type: ignore
from utils.models_utils.summa_utils import SummaPreProcessor_spatial # type: ignore
from utils.models_utils.mizuroute_utils import MizuRoutePreProcessor

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Check configurations

Now we should print our configuration settings and make sure that we have defined all the settings we need. 

In [2]:
config_path = Path('../../0_config_files/config_active.yaml')
with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)
    print(f"FORCING_DATASET: {config['FORCING_DATASET']}")
    print(f"EASYMORE_CLIENT: {config['EASYMORE_CLIENT']}")
    print(f"FORCING_VARIABLES: {config['FORCING_VARIABLES']}")
    print(f"EXPERIMENT_TIME_START: {config['EXPERIMENT_TIME_START']}")
    print(f"EXPERIMENT_TIME_END: {config['EXPERIMENT_TIME_END']}")

FORCING_DATASET: ERA5
EASYMORE_CLIENT: easymore cli
FORCING_VARIABLES: longitude,latitude,time,LWRadAtm,SWRadAtm,pptrate,airpres,airtemp,spechum,windspd
EXPERIMENT_TIME_START: 2008-01-01 00:00
EXPERIMENT_TIME_END: 2022-12-31 23:00


## Define default paths

Now let's define the paths to data directories before we run the pre processing scripts and create the containing directories

In [3]:
# Main project directory
data_dir = config['CONFLUENCE_DATA_DIR']
project_dir = Path(data_dir) / f"domain_{config['DOMAIN_NAME']}"

# Data directoris
model_input_dir = project_dir / f"{config['HYDROLOGICAL_MODEL']}_input"

# Make sure the new directories exists
model_input_dir.mkdir(parents = True, exist_ok = True)

## Create model configuration files

In [23]:
# Initialize model specific preprocessors

if config['HYDROLOGICAL_MODEL'] == 'SUMMA':
    ssp = SummaPreProcessor_spatial(config, logger)
    ssp.run_preprocessing()

    if config['DOMAIN_DEFINITION_METHOD'] != 'lumped': # lumped domain definition has no routing
        mp = MizuRoutePreProcessor(config,logger)
        mp.run_preprocessing()
    
    
elif config['HYDROLOGICAL_MODEL'] == 'FLASH':
    ssp = flashPreProcessor(config, logger)

2025-03-28 09:23:23,937 - INFO - Starting SUMMA spatial preprocessing
2025-03-28 09:23:23,938 - INFO - Starting forcing data processing
2025-03-28 09:23:23,939 - INFO - Starting to apply temperature lapse rate and add data step
2025-03-28 09:23:23,938 - INFO - Starting forcing data processing
2025-03-28 09:23:23,939 - INFO - Starting to apply temperature lapse rate and add data step
2025-03-28 09:23:23,948 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolverine_ERA5_merged_200801.nc
2025-03-28 09:23:24,222 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolverine_ERA5_merged_200802.nc
2025-03-28 09:23:24,315 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolverine_ERA5_merged_200803.nc
2025-03-28 09:23:24,412 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolverine_ERA5_merged_200804.nc
2025-03-28 09:23:24,522 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolverine_ERA5_merged_200805.nc
2025-03-28 09:23:24,612 - INFO - Processing Wolverine_ERA5_remapped_domain_Wolv

## Make Glacier attibutes and initial conditions (cold state) files

In [18]:
# basic functions
import numpy as np
import netCDF4 as nc4
import shutil
import rasterio
import geopandas as gpd
from rasterio.transform import xy
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS

def getNetCDFData(fn, varname):
    """Read <varname> variables available to be mapped from NetCDF <fn> """
    f = nc4.Dataset(fn,'r')
    data = f.variables[varname][:]
    f.close()
    return data

def getOutputPolyIDs(nc_file):
    outPolyIDs  = getNetCDFData(nc_file, 'hruId')    
    hru_elev = getNetCDFData(nc_file, 'elevation')
    hru_area = getNetCDFData(nc_file, 'HRUarea')
    gruIDs = getNetCDFData(nc_file, 'gruId')
    hru2gru = getNetCDFData(nc_file, 'hru2gruId')
    print("read data from attribute file")
    return outPolyIDs,hru_elev,hru_area, gruIDs,hru2gru

# write gru, variables to netcdf output file
def writeNC_state_vars_GRU(nc_out, newVarName, newVarType, newVarVals):
    """ Write <vars>[gru] array in netCDF4 file,<fn> and variable of
        <varname> """
    print("adding attribute data")
    ncvar = nc_out.createVariable(newVarName, newVarType, ('gru',),fill_value='-999')    
    ncvar[:] = newVarVals   # store data in netcdf file

# write gru nonscalar variables to netcdf output file
def writeNC_state_vars_GRU_VEC(nc_out, newVarName, newVarDim, newVarType, newVarVals):
    """ Write <vars>[gru] array in netCDF4 file,<fn> and variable of
        <varname> """
    print("adding GRU_VEC data")
    if newVarType=='i4' or newVarType=='i8':
        ncvar = nc_out.createVariable(newVarName, newVarType, (newVarDim,'gru',),fill_value='-999')  
    else:
        ncvar = nc_out.createVariable(newVarName, newVarType, (newVarDim,'gru',),fill_value='-999.0')  
    ncvar[:] = newVarVals   # store data in netcdf file

# write gru grid variables to netcdf output file
def writeNC_state_vars_GRU_GRID(nc_out, newVarName, newVarDim1,newVarDim2, newVarType, newVarVals):
    """ Write <vars>[gru] array in netCDF4 file,<fn> and variable of
        <varname> """
    print("adding GRU_GRID data")
    if newVarType=='i4' or newVarType=='i8':
        ncvar = nc_out.createVariable(newVarName, newVarType, (newVarDim1,newVarDim2,'grid','gru',),fill_value='-999')    
    else:
        ncvar = nc_out.createVariable(newVarName, newVarType, (newVarDim1,newVarDim2,'grid','gru',),fill_value='-999.0')    
    ncvar[:,:] = newVarVals   # store data in netcdf file

# write dom, hru, variables to netcdf output file
def writeNC_state_vars_HRU_DOM(nc_out, newVarName, newVarDim, newVarType, newVarVals):
    """ Write <vars>[hru dom] array in netCDF4 file,<fn> and variable of
        <varname> """
    print("adding HRU_DOM data")
    ncvar = nc_out.createVariable(newVarName, newVarType, (newVarDim,'hru','dom',),fill_value='-999.0')   
    ncvar[:] = newVarVals   # store data in netcdf file

# write dimensions and dimension variables to netcdf output file
def writeNC_dimsGRU(fn, grus, hru_type, glac, nx, ny):    
    """ Write <vars> array in netCDF4 file,<fn> and variable of
        <varname> """
    print("writing output file")
    nc_out = nc4.Dataset(fn, mode='w', format='NETCDF4')
    # Create dimensions
    dim_gru = nc_out.createDimension('gru', len(grus))
    dim_ngl = nc_out.createDimension('grid', glac) # max number of glaciers in any GRU
    dim_nx = nc_out.createDimension('xgrid', nx) # max number of cells in glacier bed
    dim_ny = nc_out.createDimension('ygrid', ny) # max number of cells in glacier bed
    # --- Create HRU ID variable (can be either int or string)
    if hru_type == 'str':
        # string HRU (need to add string length)
        max_strlen = 20  # EC
        dim_str = nc_out.createDimension('strlen', max_strlen)
        gruId = nc_out.createVariable('gruId', 'S1', ('gru', 'strlen'),fill_value='-999')
        gruId[:] = nc4.stringtochar(np.asarray(np.unique(grus),
                                  dtype='S{}'.format(max_strlen)))
    elif hru_type == 'int64':
        # integer HRU
        gruId = nc_out.createVariable('gruId', 'i8', ('gru', ),fill_value='-999')
        gruId[:] = grus
    elif hru_type == 'int':
        # integer HRU
        gruId = nc_out.createVariable('gruId', 'int', ('gru', ),fill_value='-999')
        gruId[:] = grus
    else:
        # not recognized
        sys.exit("ERROR, hru_type not recognized: must be str, int64, or int")
    # add attribute    
    gruId.long_name = 'GRU ID'
    return nc_out # leave netcdf file open

    # write dimensions and dimension variables to netcdf output file
def writeNC_dims(fn,  scalarv, midSoil, midToto, ifcToto, hrus, grus, hru_type, ndom, glac):    
    """ Write <vars>[hru] array in netCDF4 file,<fn> and variable of
        <varname> """
    print("writing output file")
    nc_out = nc4.Dataset(fn, 'w', format='NETCDF4')
    # Create dimensions
    dim_hru = nc_out.createDimension('hru', len(hrus))
    dim_gru = nc_out.createDimension('gru', len(grus))
    dim_scalarv = nc_out.createDimension('scalarv', scalarv)
    dim_midSoil = nc_out.createDimension('midSoil', midSoil)
    dim_midToto = nc_out.createDimension('midToto', midToto)
    dim_ifcToto = nc_out.createDimension('ifcToto', ifcToto)    
    dim_ndom = nc_out.createDimension('dom', ndom) # max number of domains in any HRU
    dim_ngl = nc_out.createDimension('glac', glac) # max number of glaciers in any GRU
    # --- Create HRU ID variable (can be either int or string)
    if hru_type == 'str':
        # string HRU (need to add string length)
        max_strlen = 20  # EC
        dim_str = nc_out.createDimension('strlen', max_strlen)
        hruId = nc_out.createVariable('hruId', 'S1', ('hru', 'strlen'),fill_value='-999')  
        hruId[:] = nc4.stringtochar(np.asarray(hrus,
                                  dtype='S{}'.format(max_strlen)))     
        gruId = nc_out.createVariable('gruId', 'S1', ('gru', 'strlen'),fill_value='-999')
        gruId[:] = nc4.stringtochar(np.asarray(np.unique(grus),
                                  dtype='S{}'.format(max_strlen)))
    elif hru_type == 'int64':
        # integer HRU
        hruId = nc_out.createVariable('hruId', 'i8', ('hru', ),fill_value='-999')   
        hruId[:] = hrus
        #hruId[:] = np.asarray(hrus, dtype='int')
        gruId = nc_out.createVariable('gruId', 'i8', ('gru', ),fill_value='-999')
        gruId[:] = grus
    elif hru_type == 'int':
        # integer HRU
        hruId = nc_out.createVariable('hruId', 'int', ('hru', ),fill_value='-999')   
        hruId[:] = hrus
        #hruId[:] = np.asarray(hrus, dtype='int')
        gruId = nc_out.createVariable('gruId', 'int', ('gru', ),fill_value='-999')
        gruId[:] = grus
    else:
        # not recognized
        sys.exit("ERROR, hru_type not recognized: must be str, int64, or int")
    # add attribute    
    hruId.long_name = 'USGS HUC12 ID'
    gruId.long_name = 'GRU ID'
    return nc_out # leave netcdf file open

In [19]:
# for making the rasters in meters
# NOTE: this could be bad since we are reprojecting a reprojected raster
#       but we needed to do the first reproject to get the correct bounding box
def reproject_to_utm(src_path, dst_path, utm_epsg):
    with rasterio.open(src_path) as src:
        # Get the CRS of the source raster
        src_crs = src.crs
        
        # Determine the UTM zone based on the provided EPSG code
        utm_crs = CRS.from_epsg(utm_epsg)
        
        # Calculate the transform and dimensions for the destination raster
        transform, width, height = calculate_default_transform(src_crs, utm_crs, src.width, src.height, *src.bounds)
        
        # Update the metadata for the destination raster
        dst_meta = src.meta.copy()
        dst_meta.update({
            'crs': utm_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        # Reproject and save the raster
        with rasterio.open(dst_path, 'w', **dst_meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src_crs,
                    dst_transform=transform,
                    dst_crs=utm_crs,
                    resampling=Resampling.nearest
                )

In [20]:
# Set up paths for grids
data_dir = config['CONFLUENCE_DATA_DIR']
domain_name = config['DOMAIN_NAME']
project_dir = Path(data_dir) / f"domain_{config['DOMAIN_NAME']}"
attribute_name = config['SETTINGS_SUMMA_ATTRIBUTES']
coldstate_name = config['SETTINGS_SUMMA_COLDSTATE']
attribute_path =  project_dir / 'settings/SUMMA'/ attribute_name
coldstate_path =  project_dir / 'settings/SUMMA'/ coldstate_name
glacier_dir = project_dir / 'attributes' / 'glaciers'

# attributes are hardwired to forcing formats (hru index rather than grid)
outPolyIDs, hru_elev, hru_area, gruIDs, hru2gru = getOutputPolyIDs(attribute_path)        
nOutPolygonsHRU = len(outPolyIDs)
nOutPolygonsGRU = len(gruIDs)
nc_outTopo_name = config['SETTINGS_SUMMA_ATTRIBUTES'][:-3] + '_glacBedTopo.nc'
nc_outTopo_path = project_dir / 'settings/SUMMA'/ nc_outTopo_name
nc_outSurf_name = config['SETTINGS_SUMMA_COLDSTATE'][:-3] + '_glacSurfTopo.nc'
nc_outSurf_path = project_dir / 'settings/SUMMA'/ nc_outSurf_name

read data from attribute file


In [21]:
# paths of rasters of glacier bed and surface topography
bed_path = project_dir / 'attributes' / 'elevation' / 'dem' / f"domain_{config['DOMAIN_NAME']}_bedrock_elv.tif"
surf_path = project_dir / 'attributes' / 'elevation' / 'dem' / f"domain_{config['DOMAIN_NAME']}_elv.tif"
debris_path = glacier_dir / f"domain_{config['DOMAIN_NAME']}_debris_thickness.tif"
hru_id_path = glacier_dir / f"domain_{config['DOMAIN_NAME']}_hru_id.tif"
id_path = glacier_dir / f"domain_{config['DOMAIN_NAME']}_glacier_id_extend.tif" # only for glacier cells, not glacierets, extended
type_path = glacier_dir / f"domain_{config['DOMAIN_NAME']}_domain_type.tif" # glacier or glacieret

# paths of reprojected rasters
temp_dir = glacier_dir / 'utm_temp'
temp_dir.mkdir(parents=True, exist_ok=True)
bed_utm_path = temp_dir / "bedrock_elv_utm.tif"
surf_utm_path = temp_dir / "elv_utm.tif"
debris_utm_path = temp_dir / "debris_thickness_utm.tif"
hru_id_utm_path = temp_dir / "hru_id_utm.tif"
id_utm_path = temp_dir / "glacier_id_utm.tif"
type_utm_path = temp_dir / "domain_type_utm.tif"

# reproject to UTM
utm_epsg = 32606 # UTM zone 6N for Alaska
reproject_to_utm(bed_path, bed_utm_path, utm_epsg)
reproject_to_utm(surf_path, surf_utm_path, utm_epsg)
reproject_to_utm(debris_path, debris_utm_path, utm_epsg)
reproject_to_utm(hru_id_path, hru_id_utm_path, utm_epsg)
reproject_to_utm(id_path, id_utm_path, utm_epsg)
reproject_to_utm(type_path, type_utm_path, utm_epsg)

# read the reprojected rasters
bed = rasterio.open(bed_utm_path)
surf = rasterio.open(surf_utm_path)
debris = rasterio.open(debris_utm_path)
hru_id = rasterio.open(hru_id_utm_path)
id = rasterio.open(id_utm_path)
type = rasterio.open(type_utm_path)

In [22]:
# for each glacier id, if entire glacier has a value in hru_id, then build a grid
nGlacier = np.zeros(nOutPolygonsGRU, dtype='i4')
basin__GlacierStorage = np.zeros(nOutPolygonsGRU, dtype='f8') # used in initial condition
max_glac = np.unique(id.read(1)).size
gridId0 = np.zeros((1, max_glac, nOutPolygonsGRU), dtype='i8')
threshold = 0.8 # threshold for glacier cells that must be in gru to be included (should be 1.0 but DEMs are not perfect)

for i in range(nOutPolygonsGRU):
    gru = gruIDs[i]
    print(f"gru: {gru}")
    hrus = outPolyIDs[hru2gru==gru]
    print(f"hrus: {hrus}")
    gru_cells = np.isin(hru_id.read(1),hrus)  
    
    # get glacier and glacieret volume in gru
    gru_surf = surf.read(1)[gru_cells]
    gru_bed = bed.read(1)[gru_cells]
    basin__GlacierStorage[i] = bed.res[0]*bed.res[1]*np.sum((gru_surf-gru_bed))*916.7e-9 # in Gt

    glac_id_cells = id.read(1)[gru_cells]  
    some_glac_in_gru = np.unique(glac_id_cells)

    # disclude id 0 and if glacier is not mostly in gru (by threshold)
    some_glac_in_gru = some_glac_in_gru[some_glac_in_gru>0]
    all_glac_in_gru = []
    for gid in some_glac_in_gru:
        if glac_id_cells[glac_id_cells==gid].size >= threshold*id.read(1)[id.read(1)==gid].size:
            all_glac_in_gru.append(gid)
    print(f"glaciers considered in GRU {i}: {all_glac_in_gru}")
    nGlacier[i] = len(all_glac_in_gru)
    gridId0[0,0:nGlacier[i],i] = all_glac_in_gru

gru: 1
hrus: [1 2 3]
glaciers considered in GRU 0: [np.int32(100332), np.int32(100333), np.int32(100334), np.int32(100570), np.int32(122903)]


In [23]:
# make arrays
glac = max(nGlacier)
gridId = gridId0[:,0:glac,:]
nx = np.zeros((1, glac, nOutPolygonsGRU), dtype='i4')
ny = np.zeros((1, glac, nOutPolygonsGRU), dtype='i4')
# dx, dy in meters since in UTM
dx = np.full((1, glac, nOutPolygonsGRU), bed.res[0], dtype='f8')
dy = np.full((1, glac, nOutPolygonsGRU), bed.res[1], dtype='f8')

# make at max size initially, then resize later
dim_nx = bed.read(1).shape[1]
dim_ny = bed.read(1).shape[0]
B0 = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='f8')
S0 = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='f8')
glacierMask0 = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='i4')
cell2hruId0  = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='i8')
type0 = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='i4')
debris_thick0 = np.zeros((dim_ny, dim_nx, glac, nOutPolygonsGRU), dtype='f8')
glacAblArea = np.zeros((glac,nOutPolygonsGRU), dtype='f8') # used in initial condition
glacAccArea = np.zeros((glac,nOutPolygonsGRU), dtype='f8') # used in initial condition
lat_moraine_wid = np.zeros((glac,nOutPolygonsGRU), dtype='f8') # used in initial condition

# get glacier cell data for each glacier
for i in range(nOutPolygonsGRU):
    for j in range(nGlacier[i]):
        
        # get glacier cells for this glacier
        glac_cells = np.where(id.read(1) == gridId[0,j,i])
        max_x = glac_cells[1].max()
        min_x = glac_cells[1].min()
        max_y = glac_cells[0].max()
        min_y = glac_cells[0].min()
        nx0 = max_x - min_x + 1 # right-left
        ny0 = max_y - min_y + 1 # top-bottom
        nx[0,j,i] = nx0
        ny[0,j,i] = ny0
        B0[:ny0,:nx0,j,i] = bed.read(1)[min_y:max_y+1,min_x:max_x+1]
        S0[:ny0,:nx0,j,i] = surf.read(1)[min_y:max_y+1,min_x:max_x+1]
        mask0 = np.zeros((dim_ny, dim_nx), dtype='i4')
        mask0[glac_cells] = 1
        glacierMask0[:ny0,:nx0,j,i] = mask0[min_y:max_y+1,min_x:max_x+1]
        type0[:ny0,:nx0,j,i] = type.read(1)[min_y:max_y+1,min_x:max_x+1]

        # equate B0 and S0 if cells are not in glacier
        B0[:ny0,:nx0,j,i][glacierMask0[:ny0,:nx0,j,i]==0] = S0[:ny0,:nx0,j,i][glacierMask0[:ny0,:nx0,j,i]==0]
        cell2hruId0[:ny0,:nx0,j,i] = hru_id.read(1)[min_y:max_y+1,min_x:max_x+1]

        # set debris thickness to 0 if not in glacier
        debris_thick0[:ny0,:nx0,j,i] = debris.read(1)[min_y:max_y+1,min_x:max_x+1]
        debris_thick0[:ny0,:nx0,j,i][glacierMask0[:ny0,:nx0,j,i]==0] = 0
        
        # sum area of glacier cells in each glacier if type is 2 (accumulation) or 3,4 (ablation)
        glacAccArea[j,i] = np.sum(bed.res[0]*bed.res[1]*glacierMask0[:ny0,:nx0,j,i][type0[:ny0,:nx0,j,i]==2])
        glacAblArea[j,i] = bed.res[0]*bed.res[1]*(np.sum(glacierMask0[:ny0,:nx0,j,i][type0[:ny0,:nx0,j,i]==3]) +
                                               np.sum(glacierMask0[:ny0,:nx0,j,i][type0[:ny0,:nx0,j,i]==4]))

# resize arrays
dim_nx = nx.max()
dim_ny = ny.max()
B = B0[:dim_ny,:dim_nx,:,:]
S = S0[:dim_ny,:dim_nx,:,:]
glacierMask = glacierMask0[:dim_ny,:dim_nx,:,:]
cell2hruId = cell2hruId0[:dim_ny,:dim_nx,:,:]
debris_thick = debris_thick0[:dim_ny,:dim_nx,:,:]

In [24]:
# create netcdf files and write dimensions
nc_outTopo = writeNC_dimsGRU(nc_outTopo_path, gruIDs, 'int', glac, dim_nx, dim_ny)
nc_outSurf = writeNC_dimsGRU(nc_outSurf_path, gruIDs, 'int', glac, dim_nx, dim_ny)

# write variables to attributes file
writeNC_state_vars_GRU(nc_outTopo, 'nGlacier', 'i4', nGlacier)
writeNC_state_vars_GRU_VEC(nc_outTopo, 'gridId', 'grid', 'i8', gridId)
writeNC_state_vars_GRU_VEC(nc_outTopo, 'nx', 'grid', 'i4', nx)
writeNC_state_vars_GRU_VEC(nc_outTopo, 'ny', 'grid', 'i4', ny)
writeNC_state_vars_GRU_VEC(nc_outTopo, 'dy', 'grid', 'f8', dy)
writeNC_state_vars_GRU_VEC(nc_outTopo, 'dx', 'grid', 'f8', dx)
writeNC_state_vars_GRU_GRID(nc_outTopo, 'bed_elev','ygrid','xgrid','f8', B)
writeNC_state_vars_GRU_GRID(nc_outTopo, 'glacierMask','ygrid','xgrid','i4', glacierMask)
writeNC_state_vars_GRU_GRID(nc_outTopo, 'cell2hruId','ygrid','xgrid','i8', cell2hruId)

# write surface elevation and debris to initial conditions file
writeNC_state_vars_GRU_VEC(nc_outSurf, 'gridId', 'grid', 'i8', gridId)
writeNC_state_vars_GRU_GRID(nc_outSurf, 'surface_elev','ygrid','xgrid','f8', S)
writeNC_state_vars_GRU_GRID(nc_outSurf, 'debris_thick','ygrid','xgrid','f8', debris_thick)

# close files
nc_outTopo.close()
nc_outSurf.close()

# remove temporary directory for reprojected UTM rasters
shutil.rmtree(temp_dir)

writing output file
writing output file
adding attribute data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_GRID data
adding GRU_GRID data
adding GRU_GRID data
adding GRU_VEC data
adding GRU_GRID data
adding GRU_GRID data


In [25]:
# set up paths for HRU and domain spatial data
nc_outAttr_name = config['SETTINGS_SUMMA_ATTRIBUTES'][:-3] + '_glac.nc'
nc_outAttr_path = project_dir / 'settings/SUMMA'/ nc_outAttr_name
nc_outInit_name = config['SETTINGS_SUMMA_COLDSTATE'][:-3] + '_glac.nc'
nc_outInit_path = project_dir / 'settings/SUMMA'/ nc_outInit_name

# for now, set nWetland to 0 (no wetlands)
nWetland = 0

In [26]:
# make new attributes file
shutil.copy(attribute_path, nc_outAttr_path)
nc_outAttr = nc4.Dataset(nc_outAttr_path, 'a')
writeNC_state_vars_GRU(nc_outAttr, 'nGlacier', 'i4', nGlacier)
nWetland = np.full((nOutPolygonsGRU), nWetland, dtype='i4')
writeNC_state_vars_GRU(nc_outAttr, 'nWetland', 'i4', nWetland)
nc_outAttr.close()

adding attribute data
adding attribute data


In [ ]:
# initialize variables at max size, then resize later
realMissing = -9999.0
ndom0 = 6 # number of possible domains
DOMarea = np.zeros((1, nOutPolygonsHRU, ndom0), dtype='f8')
DOMelev = np.full((1, nOutPolygonsHRU, ndom0), realMissing, dtype='f8')
domType = np.zeros((1, nOutPolygonsHRU, ndom0), dtype='i4')
debris_thick = np.zeros(nOutPolygonsHRU, dtype='f8')

nSnow0 = np.zeros((nOutPolygonsHRU, ndom0+1), dtype='f8')
nLake0 = np.zeros((nOutPolygonsHRU, ndom0+1), dtype='f8')
nSoil0 = np.zeros((nOutPolygonsHRU, ndom0+1), dtype='f8')
nGlce0 = np.zeros((nOutPolygonsHRU, ndom0+1), dtype='f8')

nGlce_glac = 5 # assume same number of ice layers for every glacier if exist
nSoil_glac = 3 # if has debris, will have this many soil layers
ice_layDepth = np.asarray([0.15, 0.45, 2.25, 7.0, 30.0])
ice_layDepth = ice_layDepth*4.0 # thick glacier, shouldn't need to be this thick
nToto_glac = nGlce_glac + nSoil_glac
iLayerHeight_glac = np.zeros((nOutPolygonsHRU,nToto_glac+1,4), dtype='f8') # 4 glacier domains

nLake_wtld = 5 # assume same number of lake layers for every lake if exist
nSoil_wtld = 3 # if has wetland, will have this many soil layers
lakeSed_layDepth = np.asarray([0.15, 0.45, 2.25])
nToto_wtld = nLake_wtld + nSoil_wtld
iLayerHeight_wtld = np.zeros((nOutPolygonsHRU,nToto_wtld+1), dtype='f8') # 1 wetland domain

In [28]:
# get hru data from intersections
# domain order/type here is 1)upland, 2)glacier accumulation, 3)glacier ablation clean, 4)glacier ablation debris, 5)wetland 6)glacierets
# RULES: Must have upland (area can be 0). If has accumulation, must have one of the ablation types, and vice versa (area can be 0).
intersect_path = project_dir / 'shapefiles' / 'catchment_intersection' / 'with_dem_domain'
intersect_name = 'catchment_with_dem_domain.shp'
intersect_hruId_var = config['CATCHMENT_SHP_HRUID']
shp_elev = gpd.read_file(intersect_path / intersect_name)
intersect_path = project_dir / 'shapefiles' / 'catchment_intersection' / 'with_domain_type'
intersect_name = 'catchment_with_domain_type.shp'
shp_area = gpd.read_file(intersect_path / intersect_name)

ndom = 0
for i, hru_id in enumerate(outPolyIDs):
    ind = 0
    shp_mask = (shp_elev[intersect_hruId_var].astype(int) == hru_id)
    shp_mask_count = (shp_area[intersect_hruId_var].astype(int) == hru_id)
    if any(shp_mask):
        for domain_type in range(1, ndom0+1):
            column = f'elv_mean_{domain_type}'
            # if the column exists with a valid value
            if column in shp_elev.columns:
                valid = True
                if shp_elev[column][shp_mask].values[0] is None: valid = False
                elif np.isnan(shp_elev[column][shp_mask].values[0]): valid = False
                elif shp_elev[column][shp_mask].values[0] < 0: valid = False
            else:
                valid = False
            if valid:
                DOMelev[0,i,ind] = shp_elev[column][shp_mask].values[0]
                DOMarea[0,i,ind] = shp_area[f'domType_{domain_type}'][shp_mask_count].values[0]
                domType[0,i,ind] = domain_type
                ind += 1
            else:
                if domain_type == 1:
                    DOMelev[0,i,ind] = realMissing
                    DOMarea[0,i,ind] = 0
                    domType[0,i,ind] = domain_type
                    ind += 1
                elif domain_type == 2: # if has a glacier ablation zone, must have accumulation zone
                    # NOTE: if has invalid but existing debris zone, will be replaced with clean ablation zone later as no debris thickness without valid debris zone
                    if 'elv_mean_3' in shp_elev.columns or 'elv_mean_4' in shp_elev.columns:
                        DOMelev[0,i,ind] = realMissing
                        DOMarea[0,i,ind] = 0
                        domType[0,i,ind] = domain_type
                        ind += 1
                elif domain_type == 3: # if has a glacier accumulation zone, must have ablation zone
                    if 'elv_mean_2' in shp_elev.columns:
                        if 'elv_mean_4' in shp_elev.columns: # check that valid
                            valid_4 = True
                            if shp_elev['elv_mean_4'][shp_mask].values[0] is None: valid_4 = False
                            elif np.isnan(shp_elev['elv_mean_4'][shp_mask].values[0]): valid_4 = False
                            elif shp_elev['elv_mean_4'][shp_mask].values[0] < 0: valid_4 = False
                        else:
                            valid_4 = False
                        if not valid_4: # if no valid debris ablation zone, then make clean ablation zone
                            DOMelev[0,i,ind] = realMissing
                            DOMarea[0,i,ind] = 0
                            domType[0,i,ind] = domain_type
                            ind += 1
            ndom = max(ndom,ind)
# multiply by area and divide count by number in all domains
for i in range(nOutPolygonsHRU):
    if np.sum(DOMarea[0,i,:]) > 0:
            DOMarea[0,i,:] = hru_area[i]*DOMarea[0,i,:]/np.sum(DOMarea[0,i,:])
        
intersect_path = project_dir / 'shapefiles' / 'catchment_intersection' / 'with_debris_thickness'
intersect_name = 'catchment_with_debris.shp'
shp = gpd.read_file(intersect_path / intersect_name)
for i, hru_id in enumerate(outPolyIDs):
    shp_mask = (shp[intersect_hruId_var].astype(int) == hru_id)
    if any(shp_mask):
        column = 'debri_mean'
        # if the column exists with a valid value
        if column in shp.columns:
            valid = True
            if shp[column][shp_mask].values[0] is None: valid = False
            elif np.isnan(shp[column][shp_mask].values[0]): valid = False
            elif shp[column][shp_mask].values[0] < 0: valid = False
        else:
            valid = False
        if valid: debris_thick[i] = shp[column][shp_mask].values[0] # leave at 0 if not valid
    if debris_thick[i]>0: # start with no snow
        nSoil0[i,4] = nSoil_glac
        nGlce0[i,4] = nGlce_glac
        for layer in range(nSoil_glac+1): # debris domain is index 2 in the glacier domains
            iLayerHeight_glac[i, layer,2] = debris_thick[i] * (layer ** 2 / nSoil_glac ** 2)
        for layer in range(nSoil_glac+1, nGlce_glac+nSoil_glac+1):
            iLayerHeight_glac[i, layer,2] = debris_thick[i] + ice_layDepth[layer-nSoil_glac-1]
    # accumulation and clean and glacieret never have soil, start with no snow
    nSoil0[i,np.asarray([2,3,6])] = 0
    nGlce0[i,np.asarray([2,3,6])] = nGlce_glac
    iLayerHeight_glac[i, 1:nGlce_glac+1,np.asarray([0,1,3])] = ice_layDepth

# stub code block fopr wetlands
intersect_path = project_dir / 'shapefiles' / 'catchment_intersection' / 'with_wetland'
intersect_name = 'catchment_with_wetland.shp'
#shp = gpd.read_file(intersect_path / intersect_name)
#for i, hru_id in enumerate(outPolyIDs):
#    shp_mask = (shp[intersect_hruId_var].astype(int) == hru_id)
#    if any(shp_mask):
#        column = 'lake_depth'
#        # if the column exists
#        if column in shp.columns:
#            lake_depth[i] = shp[column][shp_mask].values[0]
#    if lake_depth[i]>0: # start with no snow
#        nSoil0[i,5] = nSoil_wtld
#        nLake0[i,5] = nLake_wtld
#        for layer in range(nLake_wtld+1):
#            iLayerHeight_wtld[i, layer] = lake_depth[i] * (layer ** 2 / nLake_wtld ** 2)
#        for layer in range(nLake_wtld+1, nLake_wtld+nSoil_wtld+1):
#            iLayerHeight_wtld[i, layer] = lake_depth[i] + soil_layDepth[layer-nLake_wtld-1]
        
# resize arrays off ndom only
DOMarea = DOMarea[:,:,:ndom]
DOMelev = DOMelev[:,:,:ndom]
domType = domType[:,:,:ndom]

In [29]:
# default values for initial conditions
states = {
    'scalarCanopyIce': 0,
    'scalarCanopyLiq': 0,
    'scalarSnowDepth': 0,
    'scalarSWE': 0,
    'scalarSfcMeltPond': 0,
    'scalarAquiferStorage': 1.0,
    'scalarSnowAlbedo': 0,
    'scalarCanairTemp': 283.16,
    'scalarCanopyTemp': 283.16,
    'mLayerTemp': 283.16,
    'mLayerVolFracIce': 0,
    'mLayerVolFracLiq': 0.2,
    'mLayerMatricHead': -1.0,
    'glacMass4AreaChange': 0, # won't be in the standard attributes file
    'dt_init': 3600 # not a state, but include here
}

# read previously created initial conditions file, these will have every HRU upland
nc_in = nc4.Dataset(coldstate_path, 'r')
iLayerHeight_upld = nc_in.variables['iLayerHeight'][:,:].data.transpose()
nSoil0[:,1] = nc_in.variables['nSoil'][:].data
nSnow0[:,1] = nc_in.variables['nSnow'][:].data
nToto_upld = iLayerHeight_upld.shape[1]-1

# get state values, update with new values if exist
for var_name, var_value in states.items():
    if var_name in nc_in.variables:
        states[var_name] = nc_in.variables[var_name][0,0].data
    else:
        print(f"WARNING: {var_name} not in standard initial conditions file")
nc_in.close()

# resize number of layer arrays
nSnow = np.zeros((1, nOutPolygonsHRU, ndom), dtype='f8')
nLake = np.zeros((1, nOutPolygonsHRU, ndom), dtype='f8')
nSoil = np.zeros((1, nOutPolygonsHRU, ndom), dtype='f8')
nGlce = np.zeros((1, nOutPolygonsHRU, ndom), dtype='f8')
for i in range(nOutPolygonsHRU):
    for j in range(ndom):
        nSnow[0,i,j] = nSnow0[i,domType[0,i,j]]
        nLake[0,i,j] = nLake0[i,domType[0,i,j]]
        nSoil[0,i,j] = nSoil0[i,domType[0,i,j]]
        nGlce[0,i,j] = nGlce0[i,domType[0,i,j]]

# get dimensions based on maximum number of layers for layer types
midSoil = int(nSoil.max())
midToto = int((nSnow + nLake + nSoil + nGlce).max())
ifcToto = midToto + 1
scalarv = 1

In [30]:
# set layer heights and depths
iLayerHeight = np.zeros((ndom, nOutPolygonsHRU, ifcToto), dtype='f8')

# fill in domain values
for i in range(nOutPolygonsHRU):
    for j in range(ndom):
        if domType[0,i,j] == 1: # upland
            iLayerHeight[j,i,:nToto_upld+1] = iLayerHeight_upld[i,:]
        elif domType[0,i,j] == 2: # glacier accumulation
            iLayerHeight[j,i,:nGlce_glac+1] = iLayerHeight_glac[i,:nGlce_glac+1,0]
        elif domType[0,i,j] == 3: # glacier clean ablation
            iLayerHeight[j,i,:nGlce_glac+1] = iLayerHeight_glac[i,:nGlce_glac+1,1]
        elif domType[0,i,j] == 4: # glacier debris ablation
            iLayerHeight[j,i,:nToto_glac+1] = iLayerHeight_glac[i,:,2]
        elif domType[0,i,j] == 5: # wetland
            iLayerHeight[j,i,:nToto_wtld+1] = iLayerHeight_wtld[i,:]
        elif domType[0,i,j] == 6: # glacieret
            iLayerHeight[j,i,:nGlce_glac+1] = iLayerHeight_glac[i,:nGlce_glac+1,3]
mLayerDepth = iLayerHeight[:,:,1:] - iLayerHeight[:,:,:-1]

# transpose
iLayerHeight = iLayerHeight.transpose()
mLayerDepth = mLayerDepth.transpose()

In [31]:
# initialize netcdf file by storing dimensions and hru variable
nc_outInit = writeNC_dims(nc_outInit_path, scalarv, midSoil, midToto, ifcToto,
                        outPolyIDs, gruIDs, 'int', ndom, glac)

# write layer variables to initial conditions file
writeNC_state_vars_HRU_DOM(nc_outInit, 'iLayerHeight', 'ifcToto', 'f8', iLayerHeight)
writeNC_state_vars_HRU_DOM(nc_outInit, 'mLayerDepth', 'midToto', 'f8', mLayerDepth)
writeNC_state_vars_HRU_DOM(nc_outInit, 'nSnow', 'scalarv', 'f8', nSnow)
writeNC_state_vars_HRU_DOM(nc_outInit, 'nLake', 'scalarv', 'f8', nLake)
writeNC_state_vars_HRU_DOM(nc_outInit, 'nSoil', 'scalarv', 'f8', nSoil)
writeNC_state_vars_HRU_DOM(nc_outInit, 'nGlce', 'scalarv', 'f8', nGlce)

# write domain variables to initial conditions file
writeNC_state_vars_HRU_DOM(nc_outInit, 'domType', 'scalarv', 'f8', domType)
writeNC_state_vars_HRU_DOM(nc_outInit, 'DOMarea', 'scalarv', 'f8', DOMarea)
writeNC_state_vars_HRU_DOM(nc_outInit, 'DOMelev', 'scalarv', 'f8', DOMelev)

# write glacier variables to initial conditions file
writeNC_state_vars_GRU_VEC(nc_outInit, 'glacAblArea', 'glac', 'f8', glacAblArea)
writeNC_state_vars_GRU_VEC(nc_outInit, 'glacAccArea', 'glac', 'f8', glacAccArea)
writeNC_state_vars_GRU_VEC(nc_outInit, 'lat_moraine_wid', 'glac', 'f8', lat_moraine_wid)
writeNC_state_vars_GRU_VEC(nc_outInit, 'glacId', 'glac', 'i8', gridId)
writeNC_state_vars_GRU(nc_outInit, 'basin__GlacierStorage', 'f8', basin__GlacierStorage)

# write state variables to initial conditions file
for var_name, var_value in states.items():
    if var_name.startswith('mLayer'):
        if var_name == 'mLayerMatricHead':
            var_value_array = np.full((midSoil, nOutPolygonsHRU, ndom), var_value, dtype='f8')
            writeNC_state_vars_HRU_DOM(nc_outInit, var_name, 'midSoil', 'f8', var_value_array) 
        else:
            var_value_array = np.full((midToto, nOutPolygonsHRU, ndom), var_value, dtype='f8')
            # put ice layers at -5 C, less air as go deeper with no liquid water, similar to Giese et al. 2020 (otherwise need to spin up >40 yrs)
            if var_name == 'mLayerTemp':
                for i in range(nOutPolygonsHRU):
                    for j in range(ndom):
                         if nGlce[0,i,j] > 0:
                            start_ice = int(nSnow[0,i,j]+nLake[0,i,j]+nSoil[0,i,j])
                            end_ice   = start_ice + int(nGlce[0,i,j])
                            var_value_array[start_ice:end_ice,i,j] = 273.16 - 5
            elif var_name == 'mLayerVolFracIce':
                for i in range(nOutPolygonsHRU):
                    for j in range(ndom):
                        if nGlce[0,i,j] > 0:
                            start_ice = int(nSnow[0,i,j]+nLake[0,i,j]+nSoil[0,i,j])
                            end_ice   = start_ice + int(nGlce[0,i,j])
                            for k in range(start_ice,end_ice):
                                var_value_array[k, i, j] = 0.90 + (0.98 - 0.90) * (k - start_ice)**2 / (end_ice - start_ice - 1)**2
            elif var_name == 'mLayerVolFracLiq':
                for i in range(nOutPolygonsHRU):
                    for j in range(ndom):
                        if nGlce[0,i,j] > 0:
                            start_ice = int(nSnow[0,i,j]+nLake[0,i,j]+nSoil[0,i,j])
                            end_ice   = start_ice + int(nGlce[0,i,j])
                            var_value_array[start_ice:end_ice,i,j] = 0.0
            writeNC_state_vars_HRU_DOM(nc_outInit, var_name, 'midToto', 'f8', var_value_array)
    else:
        var_value_array = np.full((1, nOutPolygonsHRU, ndom), var_value, dtype='f8')
        writeNC_state_vars_HRU_DOM(nc_outInit, var_name, 'scalarv', 'f8', var_value_array)

# close file
nc_outInit.close()

writing output file
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_VEC data
adding GRU_VEC data
adding attribute data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
adding HRU_DOM data
